In [25]:
# IMPORTS

# Import bibliothèque de manipulation de dataframe
import pandas as pd

# Import des bibliothèques de viz
import matplotlib.pyplot as plt
import seaborn as sns

# Import split data
from sklearn.model_selection import train_test_split

# Import modèles de ML Supervisé Régression
from sklearn.linear_model import LinearRegression

# Import modèles de ML Supervisé Classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Import modèle de ML NON Supervisé
from sklearn.neighbors import NearestNeighbors

# Import des métriques
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

# Import outil standardisation de la donnée
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Import pipeline
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer


In [26]:
import nltk
from nltk.corpus import stopwords, movie_reviews
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import matplotlib.pyplot as plt
import spacy



In [27]:
nlp = spacy.load('en_core_web_sm')

In [28]:
df = pd.read_csv('../Datasets_cleaning/dataset_final_splitted.csv')

In [29]:
df_ml = df.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0']).copy()

In [30]:
df_ml.iloc[[8340]]

,Unnamed: 0.2,title,release_year,runtimeMinutes,actors,vote_average,vote_count,popularity,poster_path,imdb_id,...,genre_3,nationality_1,nationality_2,nationality_3,actors_list,actor_1,actor_2,actor_3,actor_4,actor_5
8340,8340,X-Men: Apocalypse,2016,144,"['James McAvoy', 'Michael Fassbender', 'Michae...",6.515,11947.0,42.116,/lRxsDK4exeEgKoXqI4zdr0Vl0yk.jpg,tt3385516,...,Sci-Fi,['US'],NaN,NaN,"['James McAvoy', 'Michael Fassbender', 'Michae...",James McAvoy,Michael Fassbender,Michael Fassbender,Jennifer Lawrence,Jennifer Lawrence


In [14]:
# STEMMER
# On instancie le Stemmer et on charge le set des stop words anglais dans une variable
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# On créé une fonction pour passer le texte en minuscule
def lower_case(text: str) -> str:
    return text.lower()

# On créé une fonction pour supprimer les balises html du texte
def remove_html_tags(text: str) ->  str:
    return re.sub(r'<.*?>', '', text)
# <h1> </p> <b> </b> <a href="http://google.com"> </a>

# On créé une fonction pour enlever les caractères spéciaux
def remove_special_char(text: str) -> str:
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# On créé une fonction pour enlever les stopwords
def remove_stopwords(text: str) -> str:
    return ' '.join([word for word in text.split() if word not in stop_words])

# On créé une fonction pour appliquer le stemmer sur le texte
def stem(text: str) -> str:
    return ' '.join([stemmer.stem(word) for word in text.split()])

# On créé une fonction qui va appliquer toutes les transformations sur le texte
def main_clean(review: str) -> str:
    review = lower_case(review)
    review = remove_html_tags(review)
    review = remove_special_char(review)
    review = remove_stopwords(review)
    review = stem(review)
    return review

In [31]:
# LEMMATIZER
stop_words = set(stopwords.words('english'))


# On créé une fonction pour passer le texte en minuscule
def lower_case(text: str) -> str:
    return text.lower()

# On créé une fonction pour supprimer les balises html du texte
def remove_html_tags(text: str) ->  str:
    return re.sub(r'<.*?>', '', text)
# <h1> </p> <b> </b> <a href="http://google.com"> </a>

# On créé une fonction pour enlever les caractères spéciaux
def remove_special_char(text: str) -> str:
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# On créé une fonction pour enlever les stopwords
def remove_stopwords(text: str) -> str:
    return ' '.join([word for word in text.split() if word not in stop_words])

# On créé une fonction pour appliquer le lemmatiser sur le texte
def lemmatize(text: str) -> str:
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# On créé une fonction qui va appliquer toutes les transformations sur le texte
def main_clean(review: str) -> str:
    review = lower_case(review)
    review = remove_html_tags(review)
    review = remove_special_char(review)
    review = remove_stopwords(review)
    review = lemmatize(review)
    return review

In [32]:
df_ml['clean_overview'] = df_ml['overview'].apply(main_clean)


In [134]:
# Sélection des features
X = df_ml[['release_year', 'runtimeMinutes', 'vote_average', 'vote_count', 'genre_1', 'nationality_1', 'title', 'actor_1', 'clean_overview']]

# Standardisation des données
# ----> Définition des colonnes selon si elles sont numériques ou catégorielles
col_num = ['release_year', 'runtimeMinutes', 'vote_average', 'vote_count']
col_cat = ['genre_1', 'nationality_1', 'actor_1']
col_text = ['clean_overview']

# ----> Définition des tranformations à apporter sur les colonnes selon leurs valeurs
transfo_num = Pipeline(steps=[('scaler', StandardScaler()),
                                ])

transfo_cat = Pipeline(steps=[('imputation', SimpleImputer(strategy='constant', fill_value='manquant')),
                                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transfo_text = Pipeline(steps=[('flatten', FunctionTransformer(lambda x: x.iloc[:,0], validate=False)),
                                ('Count vectorizer', TfidfVectorizer())])

# ----> Définition du process de transformation
preprocessor = ColumnTransformer([('num', transfo_num, col_num),
                                  ('cat', transfo_cat, col_cat),
                                  ('text', transfo_text, col_text)])

# Construction du pipeline global avec préprocessing et régression
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Transformation des données
    ('neighbor', NearestNeighbors(n_neighbors=6, metric='euclidean', algorithm='auto'))])  # Modèle de régression pour imputation

# Entraînement du modèle
model_pipeline.fit(X)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['release_year',
                                                   'runtimeMinutes',
                                                   'vote_average',
                                                   'vote_count']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(fill_value='manquant',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['genre_1', 'nationality_1',
                                                   'actor_1']),
                                                 ('text',
                                                  Pipeline(steps=[('flatten',
                                                                   FunctionTransformer(func=<function <lambda> at 0x0000022D56B55BC0>)),
                                                                  ('Count '
                                                                   'vectorizer',
                                                                   TfidfVectorizer())]),
                                                  ['clean_overview'])])),
                ('neighbor',
                 NearestNeighbors(metric='euclidean', n_neighbors=6))])

In [143]:
# Choix d'un film de référence
film_ref = X.loc[[4539]]  # double crochet pour garder un DataFrame

In [144]:
# transformer les données pour qu'elles soient standardisées
X_transformed = model_pipeline.named_steps['preprocessor'].transform(X)
film_transformed = model_pipeline.named_steps['preprocessor'].transform(film_ref)

# trouver les 5 films voisins
distances, indices = model_pipeline.named_steps['neighbor'].kneighbors(film_transformed)

In [145]:
# Afficher les indices des films les plus proches (hors film lui-même)
print("Indices des 5 films les plus proches :", indices[0])
print("Distances :", distances[0])

Indices des 5 films les plus proches : [4539 8594 4521 7691 6024 5445]
Distances : [0.         2.60233827 2.77222114 2.7762526  2.84969984 2.91388854]


In [146]:
# Récupérer les lignes correspondantes dans df_ml
films_proches = df_ml.iloc[indices[0]]
films_proches[['title', 'release_year', 'genre_1', 'nationality_1', 'actors']]

,title,release_year,genre_1,nationality_1,actors
4539,Fight Club,1999,Crime,['US'],"['Brad Pitt', 'Edward Norton', 'Meat Loaf', 'Z..."
8594,Avengers: Infinity War,2018,Action,['US'],"['Robert Downey Jr.', 'Robert Downey Jr.', 'Ch..."
4521,The Matrix,1999,Action,['US'],"['Keanu Reeves', 'Laurence Fishburne', 'Carrie..."
7691,Guardians of the Galaxy,2014,Action,['US'],"['Chris Pratt', 'Vin Diesel', 'Bradley Cooper'..."
6024,The Avengers,2012,Action,['US'],"['Robert Downey Jr.', 'Robert Downey Jr.', 'Ch..."
5445,Iron Man,2008,Action,['US'],"['Robert Downey Jr.', 'Gwyneth Paltrow', 'Terr..."


In [79]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mimia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mimia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True